In [ ]:
import langchain
print(langchain.__version__)

✓ LangChain 1.1.1 успешно импортирован


In [ ]:
import os, sys
from dotenv import load_dotenv

from telethon import TelegramClient, errors
import asyncio
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


nltk.download("stopwords")
sw = stopwords.words("russian")

load_dotenv()

api_id = os.getenv('APP_ID_TG')
api_hash = os.getenv('API_HASH_TG')
channels = [
    "@tass_agency",
    "@rian_ru",
    "@kommersant",
    "@gazeta_ru",
    "@meduzalive",
    "@rbc_news"
]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1 Подготовка данных

In [13]:
def clean_text_ru(text): 
   # Приведение текста к нижнему регистру
    text = text.lower()
    # Замена всех не-словесных символов на пробел (кроме букв и знаков препинания)
    text = re.sub(r'\W+', ' ', text)

    # Удаление URL-адресов
    text = re.sub(r"http\S+", "", text)

    # Создание шаблона для HTML-тегов
    html = re.compile(r'&lt;.*?&gt;')

    # Удаление HTML-тегов из текста
    text = html.sub(r'', text)

    # Список пунктуаций для удаления
    punctuations = '@#!?+&amp;*[]-%.:/();$=&gt;&lt;|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p, '')  # Удаление пунктуации

    # Удаление стоп-слов и приведение слов к нижнему регистру
    text = [word.lower() for word in text.split() if word.lower() not in sw]

    # Объединение слов обратно в текст
    text = " ".join(text)

    # Создание шаблона для поиска эмодзи
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # эмоции
                        u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                        u"\U0001F680-\U0001F6FF"  # транспорт и карты
                        u"\U0001F1E0-\U0001F1FF"  # флаги
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)

    # Удаление эмодзи из текста
    text = emoji_pattern.sub(r'', text)

    return text

In [ ]:
client = TelegramClient("session_news", api_id, api_hash)


async def load_telegram_news(channel, limit=100):
    """
    Загружает последние сообщения из указанного Telegram-канала.
    """
    try:
        print(f"Подключение к Telegram для обработки канала {channel}...")
        await client.start()

        docs = []
        print(f"Загрузка сообщений из канала {channel}...")
        async for msg in client.iter_messages(channel, limit=limit):
            if not msg.text:
                continue

            # Очистка текста
            cleaned_text = clean_text_ru(msg.text)
            # print(f"Оригинальный текст: {msg.text[:50]}...")
            # print(f"Очищенный текст: {cleaned_text[:50]}...")

            docs.append({
                "id": msg.id,
                "channel": channel,
                "text": cleaned_text,
                "date": msg.date.isoformat()
            })
        print(f"Загрузка из канала {channel} завершена. Получено {len(docs)} сообщений.")
        return docs
    except errors.UsernameInvalidError:
        print(f"Канал {channel} недоступен или не существует.")
        return []
    except errors.FloodWaitError as e:
        print(f"Превышен лимит запросов для канала {channel}. Ожидание {e.seconds} секунд.")
        await asyncio.sleep(e.seconds)
        return []
    except Exception as e:
        print(f"Ошибка при обработке канала {channel}: {e}")
        return []
    finally:
        print(f"Отключение от Telegram после обработки канала {channel}...")
        await client.disconnect()


def prepare_rag_documents(messages):
    """
    Преобразует сообщения в формат, подходящий для RAG.
    """
    print("Подготовка данных для RAG...")
    print(f"Количество сообщений для подготовки: {len(messages)}")  # Отладка
    rag_docs = [
        {
            "id": msg["id"],
            "content": msg["text"],
            "metadata": {
                "source": "telegram",
                "channel": msg["channel"],
                "date": msg["date"]
            }
        }
        for msg in messages
    ]
    print(f"Подготовлено {len(rag_docs)} документов.")
    return rag_docs


async def main():
    all_messages = []
    for channel in channels:
        print(f"Обработка канала {channel}...")
        messages = await load_telegram_news(channel, limit=100)
        print(f"Сообщений из канала {channel}: {len(messages)}")  # Отладка
        all_messages.extend(messages)

    if all_messages:
        print("Подготовка новостей для RAG...")
        rag_docs = prepare_rag_documents(all_messages)
        print("Пример подготовленного документа:")
        print(rag_docs[0] if rag_docs else "Нет документов для отображения.")  # Отладка
        return all_messages, rag_docs
    else:
        print("Нет доступных сообщений.")
        return [], []


# Запуск main() и получение данных
all_messages, rag_docs = await main()
print(f"Всего сообщений: {len(all_messages)}")  # Отладка
print(f"Всего документов для RAG: {len(rag_docs)}")  # Отладка

Обработка канала @tass_agency...
Подключение к Telegram для обработки канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Загрузка сообщений из канала @tass_agency...
Загрузка из канала @tass_agency завершена. Получено 89 сообщений.
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 89
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Загрузка из канала @tass_agency завершена. Получено 89 сообщений.
Отключение от Telegram после обработки канала @tass_agency...
Сообщений из канала @tass_agency: 89
Обработка канала @rian_ru...
Подключение к Telegram для обработки канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Загрузка сообщений из канала @rian_ru...
Загрузка из канала @rian_ru завершена. Получено 90 сообщений.
Отключение от Telegram после обработки канала @rian_ru...
Сообщений из канала @rian_ru: 90
Обработка канала @kommersant...
Подключение к Telegram для обработки канала @komme

In [25]:
rag_docs[:3]

[{'id': 351598,
  'content': 'eer другие мессенджеры используются влияния молодежь поэтому власти должны уметь работать этими инструментами заявил путин интервью телеканалу indi ody',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-04T18:14:23+00:00'}},
 {'id': 351597,
  'content': 'имеет конец бог вечен владимир путин ответил интервью indi ody вопрос реальности бессмертия российский лидер выразил уверенность реальности продления длительности человеческой жизни однако отметил всему приходит конец',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-04T17:56:23+00:00'}},
 {'id': 351596,
  'content': 'путин назвал своим внутренним правилом делать имеет права делать рассказал интервью indi ody подпишись тасс x x ru ssency',
  'metadata': {'source': 'telegram',
   'channel': '@tass_agency',
   'date': '2025-12-04T17:41:09+00:00'}}]

### 2 Подготовка поисквого индекса

In [17]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=[
        "\n\n",
        "\n",
        ". ",
        "! ",
        "? ",
        ", ",
        " ",
        ""
    ]
)

def to_langchain_docs(raw_list):
    docs = []
    for item in raw_list:
        docs.append(
            Document(
                page_content=item["content"],
                metadata=item["metadata"]
            )
        )
    return docs

def split_docs(rag_docs):
    chunks = splitter.split_documents(rag_docs)
    return chunks

In [18]:
docs = to_langchain_docs(rag_docs)
chunks = split_docs(docs)

In [24]:
chunks[:3]

[Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-04T17:19:01+00:00'}, page_content='верховный суд рф привлек дочь певицы ларисы долиной ангелину несовершеннолетнюю внучку качестве заинтересованных лиц споре вокруг квартиры заседание делу состоится 16 декабря сообщили тасс высшей судебной инстанции'),
 Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-04T17:10:25+00:00'}, page_content='памятник боевому братству кндр россии создадут курской области поддержке минобороны рф сообщил губернатор хинштейн'),
 Document(metadata={'source': 'telegram', 'channel': '@tass_agency', 'date': '2025-12-04T16:47:57+00:00'}, page_content='путин заявил восстановление ссср исключено бессмысленно отметил интервью indi ody стал искать виноватых распаде ссср дело системе')]

### 3 Подготовка поискового движка

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document





### 4 Embedder

In [21]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

c:\Users\ASUS\Documents\NewsHound\venv_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ASUS\Documents\NewsHound\venv_rag\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate d

KeyboardInterrupt: 